# Intrinsic persistent homology

This notebook contains a comparison of different distances to be considered when computing persistent homology of a point cloud assumed to be sample points _close_ to a **manifold** in a high dimensional Euclidean space.

## Generation of the point cloud: the trefoil knot
We have a function to generate a noisy sample of the trefoil knot.

In [ ]:
import numpy as np

def trefoil(n, var):
    '''
    n: size of the sample,
    var: variance of the noise with normal distribution
    '''
    phi = np.linspace(0,2*np.pi,n)
    
    X = np.sin(phi)+2*np.sin(2*phi)  + np.random.normal(0, var, n)
    Y = np.cos(phi)-2*np.cos(2*phi)  + np.random.normal(0, var, n)
    Z = - np.sin(3*phi)              + np.random.normal(0, var, n)
    
    sample_trefoil = np.column_stack((X,Y,Z))
    
    return sample_trefoil

In [ ]:
from gtda.plotting import plot_point_cloud

np.random.seed(1212)
point_cloud = trefoil(1500, 0.18)
plot_point_cloud(point_cloud)

Notice that although the trefoil knot is a _non-trivial_ embedding of $S^1$ in $\mathbb{R}^3$, it is homeomorphic to a circle. In particular, its shares the same homology groups and so its Betti numbers are $\beta_0 = 1$, $\beta_1 = 1$ and $\beta_i = 0$ for $i>1$.

## (Intrinsic) metric structure on top of the point cloud

* **Euclidean distance**

Given a sample $\mathbb{X_n}$ of points in the Euclidean space, the Euclidean distance is the most used and easily computable input distance when computing persistence diagrams. However, it does not capture the intrinsic information of the underlying topological space, being the output diagram highly dependent on the particular embedding of the data.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from gtda.plotting import plot_heatmap

E_matrix = euclidean_distances(point_cloud, point_cloud)
plot_heatmap(E_matrix, colorscale='blues')

* **kNN distance**

Given the point cloud, compute kNN-distances for a given integer parameter $k>0$ as follows:

$$ d_{kNN}(x,y) = \displaystyle \inf_{\gamma}\sum_{i}|x_i-x_{i-1}|$$
    
where the infimum is over all finite paths $\gamma = (x_0 = x, x_1, \dots, x_{r-1}, x_r = y)$ between $x$ and $y$ over the kNN graph over the point cloud.
    
In the foundational [article](https://www.science.org/doi/10.1126/science.290.5500.2319) of the popular method ISOMAP, the authors show that if the sample belongs to a Riemannian manifold $\mathcal M$ embedded in the Euclidean space, then $d_{kNN}(x,y)$ converges to the geodesic distance $d_{\mathcal M}(x,y)$. However, this metric is highly sensitive to noise.

In [ ]:
from gtda.graphs import KNeighborsGraph, GraphGeodesicDistance

k = 7
kNN = KNeighborsGraph(n_neighbors=k, mode = 'distance')
X_kNN = kNN.fit_transform(point_cloud[None,:,:])
GGD = GraphGeodesicDistance()
kNN_matrix = GGD.fit_transform(X_kNN)
plot_heatmap(kNN_matrix[0], colorscale='blues')

We can notice that the kNN-distance is far different from the Euclidean distance, specially between points that are close in the embedding of the curve in the Euclidean space but far away with respect to the geodesic distance. We compute below the MDS embedding of the pointcloud according to the kNN-distance matrix.

In [ ]:
from sklearn.manifold import MDS

embedding = MDS(n_components=3, dissimilarity='precomputed')
embedding_kNN = embedding.fit_transform(kNN_matrix[0])
plot_point_cloud(embedding_kNN)

We can observe that the presence of noise in the bottleneck regions makes the estimation of the gedesic distance quite inestable (indeed, the embedding has an _eight shape_ instead of a circle shape).

* **Fermat distance**

In presence of **noise**, it may be useful to take into account the underlying density that produces the sample. 

Given the point cloud $\mathbb{X}_n$, compute **Fermat distances** for a given  parameter $p>1$ as follows:

$$ d_{\mathbb{X}_n,p}(x,y) = \displaystyle \inf_{\gamma}\sum_{i}|x_i-x_{i-1}|^p$$
    
where the infimum is over all finite paths $\gamma = (x_0 = x, x_1, \dots, x_{r-1}, x_r = y)$ between $x$ and $y$ over the complete graph over the point cloud.
    
In this [article](https://arxiv.org/abs/2012.07621), the authors proved that if the point cloud belongs to a $d$-dimensional Riemannian manifold $\mathcal M$ embedded in a higher dimensional Euclidean space and the sample is produced according to a positive density $f:\mathcal M\to \mathbb{R}$, then $d_{p}(x,y)$ converges (modulo a reescaling factor) to a deformed geodesic distance $$d_{\mathcal M, f, p}(x,y) = \inf_{\gamma} \int_{\gamma}\frac{1}{f^{(p-1)/d}}$$
where the infimum is over all smooth paths $\gamma : [0,1]\to \mathcal M$ between $x$ and $y$ over the manifold.
Notice that $d_{p}(x,y)$ penalizes areas of low density.

It can be shown that if the geodesics in $\mathbb{X}_n$ are computed over the kNN-graph for $k = O(\log(n))$, then there is also convergence with high probability of the Fermat distance $d_{\mathbb{X}_n, p}$ towards the deformed geodesic  distance $d_{\mathcal M, f, p}$

Notice that Fermat distance is a generalization of both the Euclidean and the kNN distance. Indeed, for $p=1$ and $k$ the size of the sample, we recover the ambient Euclidean distance. On the other hand, for $p=1$ and a smaller value of $k$ we recover the kNN-distance.

In [ ]:
from gtda.graphs import FermatDistance

k = int(np.log(len(point_cloud)))
kNN = KNeighborsGraph(n_neighbors = k, mode = 'distance')
X_kNN = kNN.fit_transform(point_cloud[None,:,:])

p = 5
FD = FermatDistance(weight = p)           
FD_matrix = FD.fit_transform_plot(X_kNN) 

In [ ]:
embedding = MDS(n_components=3, dissimilarity='precomputed')
embedding_FD = embedding.fit_transform(FD_matrix[0])
plot_point_cloud(embedding_FD)

We observe now that the MDS embedding of the pointcloud with respect to the Fermat distance is a circle, with some outliers that correspond to the 
noisy points on areas of low density.

## Persistent Homology

Although different Riemannian metrics does not change the topology of a compact Riemmanian manifold viewed as a metric space, the choice of the input distance when computing persistent homology from a sample plays a central role. In what follows, we will how the choice of different metrics in the sample affects the information produced by the associated persistent diagram, in the example of the trefoil knot.

In [ ]:
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram

In [ ]:
homology_dimensions = (0, 1)

- **Euclidean distance**

In [ ]:
VR_E = VietorisRipsPersistence(homology_dimensions=homology_dimensions, metric = 'euclidean')
diagram_E = VR_E.fit_transform(point_cloud[None, :, :])
VR_E.plot(diagram_E)

The persistence diagram computed using the Euclidean distance shows many salient generators for the first homology group, as consequence of areas of small reach in the embedding of $S^1$ in $\mathbb{R}^3$. In order to capture intrinsic information of the underlying manifold (less dependent on the particular embedding), it is more desirable to endow the sample with an estimator of an intrinsic distance.

- **kNN distance**

In [ ]:
VR_kNN = VietorisRipsPersistence(homology_dimensions=homology_dimensions, metric = 'precomputed')
diagram_kNN = VR_kNN.fit_transform(kNN_matrix)
VR_kNN.plot(diagram_kNN)

The persistence diagram computed using the kNN distance shows two salient generators for the first homology group, as consequence of the wrong estimation of distances in noisy areas of small reach.

- **Fermat distance**

In [ ]:
VR_Fermat = VietorisRipsPersistence(homology_dimensions=homology_dimensions, metric = 'precomputed')
diagram_Fermat = VR_Fermat.fit_transform(FD_matrix)
VR_Fermat.plot(diagram_Fermat)

The persistence diagram computed using Fermat distance shows correctly a single generator for the first homology group. 

#### **The effect of deformation of Fermat distance with respect to $p$**

The value of the $p$ plays a relevant role in the computation of Fermat distance, since it quantifies the effect of deformation derived from the density.

In [ ]:
def Riemmanian_deformation(point_cloud, p, k, n_components):
    kNN = KNeighborsGraph(n_neighbors=k, mode = 'distance')
    X_kNN = kNN.fit_transform(point_cloud[None,:,:])
    
    FD = FermatDistance(weight = p)          
    FD_matrix = FD.fit_transform(X_kNN)
    
    embedding = MDS(n_components=n_components, dissimilarity='precomputed')
    embedding_pc = embedding.fit_transform(FD_matrix[0])
    return embedding_pc

* **Fermat deformation for $p=3$**

In [ ]:
p = 3
k = int(np.log(len(point_cloud)))
deformed_pc = Riemmanian_deformation(point_cloud, p, k, 3)
plot_point_cloud(deformed_pc)

 * **Fermat deformation for $p=4$**

In [ ]:
p = 4
k = int(np.log(len(point_cloud)))
deformed_pc = Riemmanian_deformation(point_cloud, p, k, 3)
plot_point_cloud(deformed_pc)

* **Fermat deformation for $p=5$**

In [ ]:
p = 5
k = int(np.log(len(point_cloud)))
deformed_pc = Riemmanian_deformation(point_cloud, p, k, 3)
plot_point_cloud(deformed_pc)

For $p=1$ and $k>0$, we recover the intrinsic deformation carried by the kNN-distance.

In [ ]:
p = 1
k = 5
deformed_pc = Riemmanian_deformation(point_cloud, p, k, 3)
plot_point_cloud(deformed_pc)

For $p=1$ and $k$ equal to the size of the sample, we recover the Euclidean distance.

In [ ]:
p = 1
k = len(point_cloud)-1
deformed_pc = Riemmanian_deformation(point_cloud, p, k, 3)
plot_point_cloud(deformed_pc)

## Robustness to outliers

The presence of outliers is another factor that strongly impacts the performance of computations of persistent homology. Whereas the accuracy of the approximation of the geodesic distance by the kNN-distance may be dramatically affected by the existence of outliers, the intrinsic information captured by the persistence diagrams using Fermat distance is reliable even for samples with outliers. Indeed, it remains unnafected for positive homology degrees.

Let's add some outliers to our original sample of the trefoil knot.

In [ ]:
n_out = 20
outliers = np.column_stack([(np.random.rand(n_out)-0.5)*5 for _ in range(3)])
point_cloud_outliers = np.concatenate((point_cloud, outliers))
plot_point_cloud(point_cloud_outliers)

In what follows, we will see how the addition of outliers does affect the information given by the persistent homology computation, for different choices of intrinsic input distances.

* **kNN-distance**

In [ ]:
k = int(np.log(len(point_cloud)))
kNN = KNeighborsGraph(n_neighbors=k, mode = 'distance')
X_kNN = kNN.fit_transform(point_cloud_outliers[None,:,:])
GGD = GraphGeodesicDistance()
kNN_matrix = GGD.fit_transform(X_kNN)

In [ ]:
VR_kNN = VietorisRipsPersistence(homology_dimensions=[1], metric = 'precomputed')
diagram_kNN = VR_kNN.fit_transform(kNN_matrix)
VR_kNN.plot(diagram_kNN)

In comparison with the case without the outliers, we can see that outliers produced even more salient generators of $H_1$ when using kNN-distance.

* **Fermat distance**

In [ ]:
p = 5
FD = FermatDistance(weight = p)          
FD_matrix = FD.fit_transform(X_kNN)

In [ ]:
VR_Fermat = VietorisRipsPersistence(homology_dimensions=[1], metric = 'precomputed')
diagram_Fermat = VR_Fermat.fit_transform(FD_matrix)
VR_Fermat.plot(diagram_Fermat)

After contrasting with the case without the outliers, it can be noticed that the persistent diagram for $H_1$ remains exactly the same when using Fermat distance as input. On the contrary, the addition of outliers to the point cloud should skyrocket the number of salient generators of $H_0$, since every outlier point is interpreted as a clear long-lasting connected component. 

## Application to the topological analysis of time series

The topological analysis of time series is strongly linked with the construction of the **time delay embedding** or **Takens embedding** (see [Topology of time series](https://giotto-ai.github.io/gtda-docs/latest/notebooks/topology_time_series.html)).
This construction is strongly dependent on two _parameters_: $d$ is the _embedding dimension_ and $\tau$ the _time delay_.
Although different choices of these parameters give rise to topologically equivalent spaces, their values are crucial to clearly determine the topology of the embedding in the Euclidean space. 

Recall that given a time series $f(t)$, the time delay embedding of $f$ with parameters $(d,\tau)$ is the function

$$
TD_{d,\tau} f : \mathbb{R} \to \mathbb{R}^{d}\,, \qquad t \to \begin{bmatrix}
           f(t) \\
           f(t + \tau) \\
           f(t + 2\tau) \\
           \vdots \\
           f(t + (d-1)\tau)
         \end{bmatrix}.
$$


If $f$ has a recurrent structure, then the image of $TD_{d,\tau}f$ should have a cyclic shape, that might be captured by the associated persistent diagram _for appropriate choices of $(d, \tau)$_. 

In presence of noise, it might be more robust to use Fermat distance instead of kNN distance.

In [ ]:
import plotly.graph_objects as go

x = np.linspace(0, 100, 5000)
f = np.cos(x) + np.cos(3 * x)
np.random.seed(123)
noise = np.random.normal(0,0.2,5000)

fig = go.Figure(data=go.Scatter(x=x, y=f+noise))
fig.update_layout(xaxis_title="Timestamp", yaxis_title="Amplitude")
fig.show()

In [ ]:
from gtda.time_series import SingleTakensEmbedding

max_embedding_dimension = 10
max_time_delay = 100
stride = 5

embedder_optima = SingleTakensEmbedding(
    parameters_type="search",
    time_delay=max_time_delay,
    dimension=max_embedding_dimension,
    stride=stride,
)

In [ ]:
f_embedded_optima = embedder_optima.fit_transform(f+noise)
print(f"Optimal embedding dimension is {embedder_optima.dimension_} and time delay is {embedder_optima.time_delay_}")
plot_point_cloud(f_embedded_optima)

In [ ]:
VR_E = VietorisRipsPersistence(homology_dimensions=[1], metric = 'euclidean')
diagram_E = VR_E.fit_transform(f_embedded_optima[None, :, :])
VR_E.plot(diagram_E, plotly_params = dict({
    "layout": {"title": {"text": f"PH using Euclidean distance"}}}))

In [ ]:
k = int(np.log(len(f_embedded_optima)))
kNN = KNeighborsGraph(n_neighbors=k, mode = 'distance')
X_kNN = kNN.fit_transform(f_embedded_optima[None,:,:])
GGD = GraphGeodesicDistance()
kNN_matrix = GGD.fit_transform(X_kNN)

VR_kNN = VietorisRipsPersistence(homology_dimensions=[1], metric = 'precomputed')
diagram_kNN = VR_kNN.fit_transform(kNN_matrix)
VR_kNN.plot(diagram_kNN, plotly_params = dict({
    "layout": {"title": {"text": f"PH using kNN distance"}}}))

In [ ]:
p = 10
FD = FermatDistance(weight = p)          
FD_matrix = FD.fit_transform(X_kNN)

VR_Fermat = VietorisRipsPersistence(homology_dimensions=[1], metric = 'precomputed')
diagram_Fermat = VR_Fermat.fit_transform(FD_matrix)
VR_Fermat.plot(diagram_Fermat, plotly_params = dict({
    "layout": {"title": {"text": f"PH using Fermat distance"}}}))

With some parameters, the topology of the delay embedding is clearly a **cycle**.

In [ ]:
embedding_dimension = 3
time_delay = 50
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=time_delay,
    dimension=embedding_dimension,
    stride=stride,
)

f_embedded = embedder.fit_transform(f+noise)
plot_point_cloud(f_embedded)

The estimation of the shape of the embedding of the noisy time series is more **stable** with respect to different choices of parameters if we use **Fermat distance**.
    

In [ ]:
def compute_PH_Fermat(time_delay, embedding_dimension, point_cloud, k, p):
    embedder = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=time_delay,
    dimension=embedding_dimension,
    stride=stride,
    )
    f_embedded = embedder.fit_transform(point_cloud)

    kNN = KNeighborsGraph(n_neighbors=k, mode = 'distance')
    X_kNN = kNN.fit_transform(f_embedded[None,:,:])

    FD = FermatDistance(weight = p)          
    FD_matrix = FD.fit_transform(X_kNN)

    VR_Fermat = VietorisRipsPersistence(homology_dimensions=[1], metric = 'precomputed')
    diagram_Fermat = VR_Fermat.fit_transform(FD_matrix)
    return VR_Fermat.plot(diagram_Fermat, plotly_params = dict({
    "layout": {"title": {"text": f"PH of Takens Embedding: delay = {time_delay}, dimension = {embedding_dimension}."}}
}))

In [ ]:
k = int(np.log(len(f)))
p = 5

In [ ]:
compute_PH_Fermat(40, 4, f+noise, k, p)

In [ ]:
compute_PH_Fermat(40, 5, f+noise, k, p)

In [ ]:
compute_PH_Fermat(40, 6, f+noise, k, p)

In [ ]:
compute_PH_Fermat(50, 3, f+noise, k, p)

In [ ]:
compute_PH_Fermat(50, 4, f+noise, k, p)

In [ ]:
compute_PH_Fermat(50, 5, f+noise, k, p)

In [ ]:
compute_PH_Fermat(60, 3, f+noise, k, p)

In [ ]:
compute_PH_Fermat(60, 4, f+noise, k, p)

In [ ]:
compute_PH_Fermat(60, 5, f+noise, k, p)

In [ ]:
compute_PH_Fermat(70, 4, f+noise, k, p)

In [ ]:
compute_PH_Fermat(70, 5, f+noise, k, p)

In [ ]:
compute_PH_Fermat(40, 7, f+noise, k, p)